In [1]:
import torch
import sys
from torch.utils.data import DataLoader
import torchvision.transforms as T
from tqdm import tqdm
import numpy as np
from mean_average_precision import MetricBuilder
from typing import Dict
import torchvision

sys.path.append('../src/')
from newspapersdataset import (
    NewspapersDataset, 
    prepare_data_for_dataloader
)
from functions import (
    from_tsv_to_list,
    collate_fn
)

sys.path.append("../../")
from image_size import get_image_size  # source: https://github.com/scardine/image_size

In [2]:
model = torch.load('../saved_models/model.pth', map_location=torch.device('cpu'))

In [3]:
parameters = {
    'channel': 1, 
    'rescale': [1000, 1000],
    'batch_size': 16,
    'shuffle': False, 
    'num_workers': 2,
    'main_dir': '/Users/alexdrozdz/Desktop/Studia/00. Seminarium magisterskie/Master_degree/',
    'image_dir': '/Users/alexdrozdz/Desktop/Studia/00. Seminarium magisterskie/scraped_photos_final/',
    'annotations_dir': '/Users/alexdrozdz/Desktop/Studia/00. Seminarium magisterskie/news-navigator/',
}

In [4]:
data_transform = T.Compose([
    T.Grayscale(num_output_channels=parameters['channel']),
    T.ToTensor(),
    T.Normalize((0.5,), (0.5,)),
    ])

In [5]:
# prepare data 
expected = from_tsv_to_list(parameters['annotations_dir']+'test-A/expected.tsv')
in_file = from_tsv_to_list(parameters['annotations_dir']+'test-A/in.tsv')
img_paths = [parameters['image_dir']+path for path in in_file]

data = prepare_data_for_dataloader(
    img_dir=parameters['image_dir'],
    in_list=in_file,
    expected_list=expected,
    bbox_format='x0y0x1y1',
    scale=parameters['rescale'],
    test=False,
    )
dataset = NewspapersDataset(
    df=data,
    images_path=img_paths,
    scale=parameters['rescale'],
    transforms=data_transform,
    test=False,
    )
dataloader = DataLoader(
    dataset,
    batch_size=parameters['batch_size'],
    shuffle=parameters['shuffle'],
    collate_fn=collate_fn,
    num_workers=parameters['num_workers'],
    )

In [6]:
# predict on cpu
device = 'cpu'
model.to(device)
torch.set_num_threads(1)
cpu_device = torch.device('cpu')

In [7]:
# predict on the test set
with torch.no_grad():
    f_out, f_tar = [], []
    for images, targets in tqdm(dataloader):
        images = list(image.to(device) for image in images)
        targets = [
            {k: v.to(device) for k, v in t.items()} for t in targets
        ]

        images = list(img.to(cpu_device) for img in images)
        targets = [{k: v.to(cpu_device) for k, v in t.items()} for t in targets]
        f_tar.append(targets)
        outputs = model(images)
        outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
        f_out.append(outputs)

100%|██████████| 14/14 [20:48<00:00, 89.17s/it]
